<a href="https://colab.research.google.com/github/Abdulkadir2516/Abdulkadir2516/blob/main/Mental_Health_Support_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.3 MB/s eta 0:00:00


In [5]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key= "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",
    model_name="llama-3.3-70b-versatile"
)
result = llm.invoke("what is lord ram?")
result.content


"Lord Rama is a major deity in Hinduism and is considered one of the most important figures in Hindu mythology. He is the seventh avatar (incarnation) of the god Vishnu and is revered as a symbol of righteousness, duty, and virtue.\n\nAccording to the Hindu epic, the Ramayana, Lord Rama was a king of Ayodhya, a city in ancient India. He was born to King Dasharatha and Queen Kausalya, and was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna.\n\nThe story of Lord Rama is as follows:\n\n* Lord Rama was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her own son, Bharata, to become the king.\n* During his exile, Lord Rama's wife, Sita, was abducted by the demon king Ravana, who took her to his kingdom in Lanka (modern-day Sri Lanka).\n* Lord Rama, along with his brother Lakshmana and the monkey god Hanuman, embarked on a journey to rescue Sita and defeat Ravana.\n* After a long and arduous battle, Lord Rama defeated Ravana and rescued